Taking the positive and negative reviews of 2 politician: Politican X and Politician Y

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Working for politician X

In [ ]:
with open('drive/My Drive/PoliticalLeaderX.txt', 'r') as f2:
    data = f2.read()
    #print(data)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sentences=nltk.sent_tokenize(data)

In [ ]:
from  textblob import TextBlob
blob = TextBlob(data.strip())

In [ ]:
import csv
header_name = ['labels', 'data']
with open('sentiment.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=header_name)
    writer.writeheader()

In [ ]:
for sent in blob.sentences:
  #print(sent.sentiment.polarity)
  with open('sentiment.csv', 'a') as file:
    pol=sent.sentiment.polarity
    if(pol<0):
      pol=-1
    else:
      pol=1
    writer = csv.DictWriter(file, fieldnames=header_name)
    info = {
                'labels': pol,
                'data': sent
            }
    writer.writerow(info)

In [ ]:
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [ ]:
import pandas as pd
df=pd.read_csv('sentiment.csv')

In [ ]:
type(df['data'][8])

str

In [ ]:
df.head

<bound method NDFrame.head of        labels                                               data
0           1  india s shock gdp growth rate is a crisis modi...
1          -1  most analysts had   belatedly   forecast the b...
2           1  it is now clear that if the government does no...
3           1  the economy is on a cusp from where it can swi...
4           1  nirmala sitharaman is on test.if we get past t...
...       ...                                                ...
14356      -1      because the advertising market is broken too.
14357       1  if you think we deserve your support  do join ...
14358       1  your support will define our journalism  and t...
14359      -1      it will take just a few seconds of your time.
14360       1            support our journalismshow full article

[14361 rows x 2 columns]>

In [ ]:
from nltk.tokenize import word_tokenize
allowed_word_types = ["J"]
all_words=[]
for p in  df['data']:
    cleaned = re.sub(r'[^(a-zA-Z)\s]','', p)
    
    tokenized = word_tokenize(cleaned)
    
    stopped = [w for w in tokenized if not w in stopwords.words('english')]
    
    pos = nltk.pos_tag(stopped)
    
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())
   

Bag of words model

In [ ]:
# creating a frequency distribution of each adjectives.
all_words = nltk.FreqDist(all_words)

# listing the 1000 most frequent words
word_features = list(all_words.keys())[:1000]

In [ ]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [ ]:
import random
dataset=[]
i=0
for p in df['data']:
  feature=find_features(p)
  dataset.append([feature,df['labels'][i]])
  i+=1
random.shuffle(dataset)

print(len(dataset))# dataset is a bag of model of vectors

training_set = dataset[:10000]
testing_set = dataset[10000:]

14361


Machine Learning Classifiers

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 86.44806237101582


In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# training various models by passing in the sklearn models into the SklearnClassifier from NLTK 

MNB_clf = SklearnClassifier(MultinomialNB())
MNB_clf.train(training_set)

BNB_clf = SklearnClassifier(BernoulliNB())
BNB_clf.train(training_set)

LogReg_clf = SklearnClassifier(LogisticRegression())
LogReg_clf.train(training_set)

SVC_clf = SklearnClassifier(SVC())
SVC_clf.train(training_set)

<SklearnClassifier(SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))>

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
ground_truth = [r[1] for r in testing_set]
predictions = []
f1_scores = {}
Li={0:MNB_clf,1:BNB_clf,2:LogReg_clf,3:SVC_clf}
print(Li[0])
for x in Li:
  predictions=[]
  for r in testing_set:
    predictions.append(Li[x].classify(r[0]))
  if(x==0):
    str='Multinomial Naive Baeyes'
  elif(x==1):
    str='Bernoulli Naive Baeyes'
  elif(x==2):
    str='Logistic Regression'
  else:
    str='Support Vector'
  print(str,'classifier accuracy :',accuracy_score(ground_truth,predictions))
  print(str,'f1 score :',f1_score(ground_truth,predictions))

<SklearnClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))>
Multinomial Naive Baeyes classifier accuracy : 0.8651685393258427
Multinomial Naive Baeyes f1 score : 0.9203899268887085
Bernoulli Naive Baeyes classifier accuracy : 0.8601238248108232
Bernoulli Naive Baeyes f1 score : 0.9170068027210885
Logistic Regression classifier accuracy : 0.8791561568447603
Logistic Regression f1 score : 0.929479459387127
Support Vector classifier accuracy : 0.8793854620499886
Support Vector f1 score : 0.9306800210859252


So we clearly see that Logistic regression classifier and Support vector perfectly classifies our dataset
But since SVM has more f1-score so we will make predictions based on SVM

In [ ]:
predictions=[]
for r in testing_set:
  predictions.append(SVC_clf.classify(r[0]))
print(predictions)


In [ ]:
positive=0
negative=0
for i in range(0,len(predictions)):
  if predictions[i]==1:
    positive=positive+1
  else:
    negative=negative+1

In [ ]:
print(positive,negative)

4030 331


Sentimental Analysis

In [ ]:
positive_percent=positive/(positive+negative)
negative_percent=negative/(positive+negative)

In [ ]:
print(positive_percent*100,negative_percent*100)

92.40999770694795 7.590002293052052


So nearly 92% people are in favour of politician X and around 8% people do not favour him

Taking new Data for another politician (politician Y)

In [ ]:
data=''
with open('drive/My Drive/PoliticalLeaderY.txt', 'r') as f2:
    data = f2.read()
    #print(data)

In [ ]:
from  textblob import TextBlob
blob = TextBlob(data.strip())

In [ ]:
header_name = ['labels', 'data']
with open('sentiment1.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=header_name)
    writer.writeheader()

In [ ]:
for sent in blob.sentences:
  #print(sent.sentiment.polarity)
  with open('sentiment1.csv', 'a') as file:
    pol=sent.sentiment.polarity
    if(pol<0):
      pol=-1
    else:
      pol=1
    writer = csv.DictWriter(file, fieldnames=header_name)
    info = {
                'labels': pol,
                'data': sent
            }
    writer.writerow(info)

In [ ]:
df=[]
import pandas as pd
df=pd.read_csv('sentiment1.csv')

In [ ]:
df.head

<bound method NDFrame.head of        labels                                               data
0           1         no cure yet for covid  or congress crisis.
1           1  rahul gandhi continuously fails to win india\n...
2           1  just as there is no single medicine to cure co...
3           1  in the absence of any vaccine  social distanci...
4           1  similarly  the need for a strong central leade...
...       ...                                                ...
19215      -1      because the advertising market is broken too.
19216       1  if you think we deserve your support  do join ...
19217       1  your support will define our journalism  and t...
19218      -1      it will take just a few seconds of your time.
19219       1            support our journalismshow full article

[19220 rows x 2 columns]>

In [ ]:
from nltk.tokenize import word_tokenize
allowed_word_types = ["J"]
all_words=[]
for p in  df['data']:
    cleaned = re.sub(r'[^(a-zA-Z)\s]','', p)
    
    tokenized = word_tokenize(cleaned)
    
    stopped = [w for w in tokenized if not w in stopwords.words('english')]
    
    pos = nltk.pos_tag(stopped)
    
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

Making Bag of words model

In [ ]:
# creating a frequency distribution of each adjectives.
all_words = nltk.FreqDist(all_words)

# listing the 1000 most frequent words
word_features = list(all_words.keys())[:1000]

In [ ]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [ ]:
import random
dataset=[]
i=0
for p in df['data']:
  feature=find_features(p)
  dataset.append([feature,df['labels'][i]])
  i+=1
random.shuffle(dataset)

print(len(dataset))# dataset is a bag of model of vectors

# training_set = dataset[:15000]
# testing_set = dataset[15000:]

19220


Applying Machine Learning Algorithm as trained earlier that is SVM



In [ ]:
predictions=[]
for r in dataset:
  predictions.append(SVC_clf.classify(r[0]))
print(predictions)

In [ ]:
ground_truth=[]
for r in dataset:
  ground_truth.append(r[1])

In [ ]:
print(str,'classifier accuracy :',accuracy_score(ground_truth,predictions))
print(str,'f1 score :',f1_score(ground_truth,predictions))

Support Vector classifier accuracy : 0.8559833506763788
Support Vector f1 score : 0.917653358719581


Sentimental Analysis

In [ ]:
positive=0
negative=0
for i in range(0,len(predictions)):
  if predictions[i]==1:
    positive=positive+1
  else:
    negative=negative+1

In [ ]:
positive_percent=positive/(positive+negative)
negative_percent=negative/(positive+negative)

In [ ]:
print(positive_percent*100,negative_percent*100)

94.15712799167534 5.842872008324662


So this politician (politician Y) has also 94% positive reviews and 6 % negative reviews

